In [ ]:
import os
import torch
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import PeftModel
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from data_loader import load_voicephishing_data
from datasets import load_dataset


# pip install --upgrade -q datasets peft transformers==4.45.2
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # 멀티 GPU 사용 시
np.random.seed(seed)
random.seed(seed)


# ===== 고정 경로 설정 =====
BASE_DIR = "/home/work/Voicephishing"
CACHE_DIR = os.path.join(BASE_DIR, "cache")
BASE_MODEL_PATH = "maywell/Synatra-42dot-1.3B"
ADAPTER_PATH = os.path.join(BASE_DIR, "model/model_synatra_cls_251021")
TOKENIZER_PATH = os.path.join(BASE_DIR, "tokenizer/tokenizer_synatra_cls_251021")
TEST_DATA_PATH = os.path.join(BASE_DIR, "dataset/korccvi_256_0_test.csv")
SAVE_PATH = os.path.join(BASE_DIR, "dataset/eval_folder/eval_synatra_cls_251021")
SCORES_PATH = os.path.join(BASE_DIR, "dataset/eval_folder/eval_metrics.csv")

# ===== 환경 변수 =====
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["TRANSFORMERS_CACHE"] = CACHE_DIR
os.environ["TRITON_CACHE_DIR"] = os.path.join(CACHE_DIR, "triton")
os.environ["HUGGINGFACE_HUB_CACHE"] = "/home/work/Voicephishing/hf_cache"

# ===== 전처리 함수 =====
def preprocess_input(text):
    text = " ".join(text.strip().split())
    return text

# ===== 평가 함수 =====
def evaluate_and_save():
    tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH, cache_dir=CACHE_DIR)
    tokenizer.pad_token = tokenizer.eos_token

    # 저장된 테스트 데이터셋 파일 로드
    test_dataset = load_dataset("csv", data_files=TEST_DATA_PATH)["train"]

    base_model = AutoModelForSequenceClassification.from_pretrained(
        BASE_MODEL_PATH,
        num_labels=2,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.bfloat16,
        cache_dir=CACHE_DIR,
        low_cpu_mem_usage=True,
        device_map="cuda:0"
    )

    model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)
    model = model.merge_and_unload()
    model.eval()
    for p in model.parameters():
        p.requires_grad = False

    records = []
    y_true = []
    y_pred = []

    label_map = {0: "정상", 1: "보이스피싱"}
    inv_label_map = {"정상": 0, "보이스피싱": 1}

    for sample in tqdm(test_dataset, desc="🧪 Evaluating"):
        input_text = sample["text"]
        target_label = label_map[sample["Label"]]

        preprocessed_text = preprocess_input(input_text)
        inputs = tokenizer(preprocessed_text, return_tensors="pt", padding="max_length", truncation=True, max_length=1024, return_token_type_ids=False)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        with torch.no_grad():
            logits = model(**inputs).logits
            predicted_label = torch.argmax(logits, dim=-1).item()
            # predicted_label = 1 if logits.item() > 0 else 0

        pred_text = label_map[predicted_label]
        is_correct = (pred_text == target_label)

        y_true.append(inv_label_map[target_label])
        y_pred.append(inv_label_map[pred_text])

        records.append({
            "Input": input_text,
            "Prediction": pred_text,
            "Label": target_label,
            "Correct": int(is_correct)
        })

    # ===== 지표 계산 =====
    acc = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    # ===== 결과 출력 =====
    scores_df = pd.DataFrame([{
        "Accuracy": acc,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1
    }])

    print("\n✅ 평가 지표:")
    print(scores_df.to_string(index=False, float_format="%.4f"))

    # ===== 결과 저장 =====
    os.makedirs(os.path.dirname(SAVE_PATH), exist_ok=True)
    pd.DataFrame(records).to_csv(SAVE_PATH, index=False, encoding="utf-8-sig")
    scores_df.to_csv(SCORES_PATH, index=False, encoding="utf-8-sig")

    print(f"\n📁 예측 결과 저장 완료: {SAVE_PATH}")
    print(f"📁 평가 지표 저장 완료: {SCORES_PATH}")

# ===== 실행 =====
if __name__ == "__main__":
    evaluate_and_save()
